<a href="https://colab.research.google.com/github/DCODE-ARMY/GAN/blob/main/MLProjectversion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=1a7e03bacffc0a15fc0a0e8f77e9f4dbf156750a82124e24121e2819e7092c60
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [ ]:
import os
import numpy as np
import pickle
import requests
import matplotlib.pyplot as plt
import wget
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.utils import shuffle
import numpy as np
from tensorflow import keras


In [ ]:
def load_coursework_data():
    import wget
    path_to_check = os.path.join('', 'image_recognition_dataset.pth')
    if not os.path.exists(path_to_check):
        print("Cannot find the dataset on local !")
        print("Downloading the dataset from cloud, it may take several mintues.")
        url = 'https://www.dropbox.com/s/j05hyf667ikzu4s/image_recognition_dataset.pth?dl=1'
        filename = wget.download(url)
        data_path = os.path.join('', filename)
    else:
        print("Data founded on local {:s}".format(path_to_check))
        data_path = path_to_check
        
    data = pickle.load(open(data_path, 'rb'))
    train_imgs,train_labels,test_imgs,test_labels = data.values()
    return train_imgs,train_labels,test_imgs,test_labels

In [ ]:
x,y,test_imgs,test_labels = load_coursework_data()
validation_x,validation_y=x[:5000],y[:5000]
# validation_x=validation_x*(1./255)

train_imgs=x[5000:]
# train_imgs=train_imgs*(1./255)
train_labels=y[5000:]

#converting labels to one hot
train_labels=np.squeeze(tf.one_hot(train_labels,depth=8))

validation_y=np.squeeze(tf.one_hot(validation_y,depth=8))
test_labels=np.squeeze(tf.one_hot(test_labels,depth=8))

Cannot find the dataset on local !


In [ ]:
# # train_labels1=tf.one_hot(train_labels,depth=8)
# # train_labels1[:4]
# train_labels.shape
# np.squeeze(train_labels).shape

In [ ]:
cls_train = len(np.unique(y))
print("Total number of image classes : {:d}".format(cls_train))
print("Shape of training set", train_imgs.shape)

print("Training image count: ", train_imgs.shape[0])
print("Test image count: ", test_imgs.shape[0])

In [ ]:
# train_dataset=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,horizontal_flip=True,zoom_range=0.2,shear_range=0.2,
#                                                               rotation_range=0.2)


In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((train_imgs,train_labels))
train_dataset=train_dataset.shuffle(buffer_size=64).batch(batch_size=64)

validation_dataset=tf.data.Dataset.from_tensor_slices((validation_x,validation_y))
validation_dataset=validation_dataset.shuffle(buffer_size=64).batch(batch_size=64)


test_dataset=tf.data.Dataset.from_tensor_slices((test_imgs,test_labels))
test_dataset=test_dataset.shuffle(buffer_size=64).batch(batch_size=64)



In [ ]:
import matplotlib.pyplot as plt
class_names=['airplane','automobile','bird','cat','deer','dog','frog','horse']
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_imgs[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[y[5000:][i][0]])
plt.show()

In [ ]:
image_size=32
channel_axis=3

In [ ]:
def model():
    inp =layers.Input(shape=[image_size, image_size,channel_axis])
    # inp =layers.Rescaling(1./255)(inp)
    inp =layers.RandomFlip('horizontal')(inp)
    inp =layers.RandomRotation(0.2)(inp)
    inp =layers.RandomZoom(0.2)(inp)
    
    #branch 1
    x=layers.Conv2D(filters=128,kernel_size=5,strides=(2,2),padding='same')(inp)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
   
    
    x=layers.Conv2D(filters=128,kernel_size=3,strides=(2,2),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
    x=layers.Conv2D(filters=256,kernel_size=3,strides=(2,2),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)

    
    x=layers.Conv2D(filters=256,kernel_size=3,strides=(2,2),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
    
    x=layers.Conv2D(filters=256,kernel_size=3,strides=(2,2),padding='same')(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Dropout(0.5)(x)
    
    # x=layers.Conv2D(filters=128,kernel_size=2,strides=(2,2),padding='same',activation='relu')(x)
    # x=layers.Dropout(0.4)(x)
    
    # x=layers.Conv2D(filters=128,kernel_size=2,strides=(2,2),padding='same',activation='relu')(x)
    # x=layers.Dropout(0.4)(x)
    
    
    
    x=layers.Flatten()(x)
    
    x=layers.Dense(units=1024,kernel_regularizer=tf.keras.regularizers.L2(0.01),
                  )(x) # activity_regularizer=tf.keras.regularizers.L2(0.01)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    x=layers.Dropout(0.5)(x)

    x=layers.Dense(units=512,kernel_regularizer=tf.keras.regularizers.L2(0.01),
                   )(x)  #activity_regularizer=tf.keras.regularizers.L2(0.01)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    # x=layers.Dropout(0.4)(x)
    
    # x=layers.Dense(units=256,kernel_regularizer=tf.keras.regularizers.L2(0.01),
    #               )(x)  # activity_regularizer=tf.keras.regularizers.L2(0.01)
    # x=layers.BatchNormalization()(x)
    # x=layers.ReLU()(x)
    # x=layers.Dropout(0.4)(x)
    
    x=layers.Dense(units=8,activation='softmax')(x)
    # x=layers.Softmax()(x)
    
    return keras.Model(inputs=[inp],outputs=x)
    
    
    
    
    

In [ ]:
model=model()
model.summary()

In [ ]:
learning_rate=0.02

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])



In [ ]:
model.load_weights('/content/drive/MyDrive/saved model/72ACC')

In [ ]:
loss, acc = model.evaluate(test_dataset, verbose=2)
print('loss ',loss,'acc ',acc)

125/125 - 8s - loss: 3.0818 - accuracy: 0.7204 - 8s/epoch - 62ms/step
loss  3.0817677974700928 acc  0.7203750014305115


In [ ]:

checkpoint_dir = '/content/drive/MyDrive/MLProject/chk'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

csv_logger=tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/MLProject/log2.csv', separator=",", append=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=False,
    monitor="val_accuracy",
    save_best_only=True,
)


model.fit(
    train_dataset,
    epochs=1000,
    validation_steps=60,
    validation_data=validation_dataset,
    callbacks=[csv_logger,
        checkpoint_callback,],
)

Epoch 1/1000
547/547 [==============================] - ETA: 0s - loss: 4.6763 - accuracy: 0.3622

547/547 [==============================] - 20s 25ms/step - loss: 4.6763 - accuracy: 0.3622 - val_loss: 2.8387 - val_accuracy: 0.4138
Epoch 2/1000
544/547 [============================>.] - ETA: 0s - loss: 2.4872 - accuracy: 0.4895

547/547 [==============================] - 10s 17ms/step - loss: 2.4862 - accuracy: 0.4898 - val_loss: 2.2840 - val_accuracy: 0.5461
Epoch 3/1000
545/547 [============================>.] - ETA: 0s - loss: 2.2822 - accuracy: 0.5390

547/547 [==============================] - 9s 17ms/step - loss: 2.2815 - accuracy: 0.5390 - val_loss: 2.2336 - val_accuracy: 0.5474
Epoch 4/1000
545/547 [============================>.] - ETA: 0s - loss: 2.1198 - accuracy: 0.5792

547/547 [==============================] - 10s 18ms/step - loss: 2.1189 - accuracy: 0.5794 - val_loss: 2.0574 - val_accuracy: 0.5836
Epoch 5/1000
543/547 [============================>.] - ETA: 0s - loss: 2.0356 - accuracy: 0.6157

...

547/547 [==============================] - 9s 16ms/step - loss: 0.8926 - accuracy: 0.9811 - val_loss: 2.5155 - val_accuracy: 0.7255
Epoch 156/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.8822 - accuracy: 0.9817 - val_loss: 3.7421 - val_accuracy: 0.6706
Epoch 157/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9349 - accuracy: 0.9813 - val_loss: 2.8379 - val_accuracy: 0.6638
Epoch 158/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9308 - accuracy: 0.9792 - val_loss: 3.3228 - val_accuracy: 0.6388
Epoch 159/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9359 - accuracy: 0.9793 - val_loss: 3.1207 - val_accuracy: 0.7057
Epoch 160/1000
547/547 [==============================] - 6s 11ms/step - loss: 0.9120 - accuracy: 0.9826 - val_loss: 2.2254 - val_accuracy: 0.6909
Epoch 161/1000
547/547 [==============================] - 6s 11ms/step - loss: 0.9924 - accuracy: 0.9804 - val_loss: 3.5649 - val_acc

547/547 [==============================] - 9s 17ms/step - loss: 0.9171 - accuracy: 0.9833 - val_loss: 2.3661 - val_accuracy: 0.7266
Epoch 202/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9408 - accuracy: 0.9840 - val_loss: 2.7818 - val_accuracy: 0.6990
Epoch 203/1000
547/547 [==============================] - 6s 11ms/step - loss: 0.8499 - accuracy: 0.9854 - val_loss: 2.6050 - val_accuracy: 0.7141
Epoch 204/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9684 - accuracy: 0.9846 - val_loss: 1.9486 - val_accuracy: 0.7096
Epoch 205/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9893 - accuracy: 0.9819 - val_loss: 2.0596 - val_accuracy: 0.7010
Epoch 206/1000
547/547 [==============================] - 6s 11ms/step - loss: 0.9356 - accuracy: 0.9829 - val_loss: 2.0693 - val_accuracy: 0.6839
Epoch 207/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9048 - accuracy: 0.9846 - val_loss: 2.6290 - val_acc

547/547 [==============================] - 9s 17ms/step - loss: 0.9764 - accuracy: 0.9847 - val_loss: 2.9098 - val_accuracy: 0.7286
Epoch 288/1000
547/547 [==============================] - 6s 12ms/step - loss: 1.0108 - accuracy: 0.9852 - val_loss: 3.1250 - val_accuracy: 0.7143
Epoch 289/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.8673 - accuracy: 0.9860 - val_loss: 2.4246 - val_accuracy: 0.6854
Epoch 290/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9509 - accuracy: 0.9858 - val_loss: 2.2131 - val_accuracy: 0.7195
Epoch 291/1000
547/547 [==============================] - 7s 12ms/step - loss: 0.9581 - accuracy: 0.9856 - val_loss: 2.3976 - val_accuracy: 0.7049
Epoch 292/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.8865 - accuracy: 0.9869 - val_loss: 1.7141 - val_accuracy: 0.7167
Epoch 293/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9490 - accuracy: 0.9874 - val_loss: 2.3318 - val_acc

547/547 [==============================] - 10s 17ms/step - loss: 0.9733 - accuracy: 0.9879 - val_loss: 3.0051 - val_accuracy: 0.7292
Epoch 584/1000
547/547 [==============================] - 6s 12ms/step - loss: 1.0184 - accuracy: 0.9896 - val_loss: 3.4452 - val_accuracy: 0.7169
Epoch 585/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9639 - accuracy: 0.9895 - val_loss: 2.1669 - val_accuracy: 0.6906
Epoch 586/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9782 - accuracy: 0.9895 - val_loss: 3.5634 - val_accuracy: 0.7076
Epoch 587/1000
547/547 [==============================] - 7s 13ms/step - loss: 1.0293 - accuracy: 0.9882 - val_loss: 3.3131 - val_accuracy: 0.7102
Epoch 588/1000
547/547 [==============================] - 6s 12ms/step - loss: 1.0151 - accuracy: 0.9883 - val_loss: 2.8739 - val_accuracy: 0.6878
Epoch 589/1000
547/547 [==============================] - 6s 12ms/step - loss: 0.9950 - accuracy: 0.9880 - val_loss: 1.9308 - val_ac

KeyboardInterrupt: ignored

In [ ]:
"""
case1:
  only model without softmax and from logits=true no regularization or batchnorm  was used
  lr=0.05 
  op:
  acc and val acc remains on 0.12.. and val_loss was nan

case 2:
  added batchNorm to each branch
  added regularization to dense layers
  op
  accuracy increased to 50 and remained close to it 

case 3:
  changed sparse to categorical 
  added dropout to ddense layers 
  chaged regularizers to L2
  added one more dense layer with 1024 units 

case 4:
  changed the architecture , filter size=128
  no batchNorm between conv layers
  one dense layer with 1024 neurons
  op:
  got 60% acc

case 5:
  increased the filter size to 256
  op:
  acc is not increased

case 6:
  introduced batchnorm between convLayers
  changed leraning rate from 0.05 to 0.02
  logcsv1 69 acc

  op
  acc is 99% and val acc is 69%

case 7:
  

"""